## Usage

Lagent 受到 PyTorch 设计理念的启发。我们期望，与神经网络层的类比能让工作流程更清晰、更直观，这样用户只需专注于以 Python 风格创建层并定义它们之间的消息传递。这是一个简单的教程，能让你快速上手构建多智能体应用程序。 

### Model as Agents 

Agents use AgentMessage for communication.

In [4]:
from typing import Dict, List 
from lagent.agents import Agent
from lagent.schema import AgentMessage
from lagent.llms import VllmModel, INTERNLM2_META

llm = VllmModel(
    path="Qwen/Qwen3-0.6B", 
    meta_template=INTERNLM2_META,
    tp=1, 
    top_k=1,
    temperature=1.0, 
    stop_words=['<|im_end|>'], 
    max_new_tokens=1024
)

system_prompt = '你的回答只能从“典”、“孝”、“急”三个字中选一个。'
agent = Agent(llm, system_prompt)

user_msg = AgentMessage(sender='user', content='今天天气情况')
bot_msg = agent(user_msg)
print(bot_msg)

ModuleNotFoundError: No module named 'vllm'